# NYC Parking Violations - Vehicle Color Standardization - Large Dataset Exmaple

This notebook contains examples to demonstrate data cleaning funcitonality in [openclean](https://github.com/VIDA-NYU/openclean) using a large dataset.

*Dataset*. The dataset that is used in this examples is the [NYC Parking Violations Issued - Fiscal Year 2014](https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2014/jt7v-77mi) dataset that contains violations issued during the respective fiscal year. The dataset is available for download via the [Socrata Open Data API](https://dev.socrata.com/). The 2014 parking violations dataset contains over 9 million rows and the data file is about 380 MB in size.

In this example we use a subset of the dataset containing five columns and about 8 million rows. We look at the problem of standardizing the values in the *Vehicle Color* column.

## Download Dataset

In [1]:
# Download the full 'NYC Parking Violations Issued - Fiscal Year 2014' dataset.
# Note that the downloaded full dataset file is about 380 MB in size! Use the
# alternative data file with 10,000 rows that is included in the repository if
# you do not want to download the full data file.

import gzip
import humanfriendly
import os

from openclean.data.source.socrata import Socrata

dataset = Socrata().dataset('jt7v-77mi')

# By default, this example uses a small sample of the full dataset that
# is included in the 'data' subfolder within this repository.
#datafile = './data/jt7v-77mi.tsv.gz'

# Remove the comment for this line if you want to use the full dataset.
datafile = './jt7v-77mi.tsv.gz'


# Download file only if it does not exist already.
if not os.path.isfile(datafile):
    with gzip.open(datafile, 'wb') as f:
        print('Downloading ...\n')
        dataset.write(f)


fsize = humanfriendly.format_size(os.stat(datafile).st_size)
print("Using '{}' in file {} of size {}".format(dataset.name, datafile, fsize))

Using 'Parking Violations Issued - Fiscal Year 2014' in file ./jt7v-77mi.tsv.gz of size 379.19 MB


## Load Dataset

Load initial snapshot of the dataset into a persistent database manager. The snapshot for this example contains a subset of the columns from the original dataset.

In [2]:
# Due to the size of the full dataset file, we make use of openclean's
# stream operator to avoid having to load the dataset into main-memory.

from openclean.function.eval.base import Col
from openclean.function.eval.logic import And
from openclean.function.eval.null import IsNotEmpty
from openclean.pipeline import stream

data = stream(datafile)\
    .select(['Plate ID', 'Registration State', 'Plate Type', 'Violation Code', 'Vehicle Color'])\
    .rename('Registration State', 'State')\
    .where(And(IsNotEmpty('Vehicle Color'), Col('State') != '99', Col('Plate ID') != '999'))\
    .update('Vehicle Color', str.upper)


In [3]:
# Count number of rows. This will also give us an idea for how
# long it takes to scan the input document.

import time

start_scan = time.perf_counter()

rows = data.count()

end_scan = time.perf_counter()

print('Scan time {:0.4f} sec. for {} rows.'.format((end_scan - start_scan), rows))

Scan time 111.4550 sec. for 8993835 rows.


In [4]:
# Create a database object that materializes the dataset and
# all provenance information in a folder on the local file system.

from openclean.engine.base import DB

db = DB(basedir='.openclean', create=True)

In [5]:
# Load snapshot into database.

import time

start_load = time.perf_counter()

db.load_dataset(source=data, name='vehicle_data')

end_load = time.perf_counter()

print('Load time {:0.4f} sec.'.format(end_load - start_load))

Load time 448.3509 sec.


In [6]:
# Print the first ten rows of the dataset to get a first
# idea of the content.

db.stream('vehicle_data').head()

,Plate ID,State,Plate Type,Violation Code,Vehicle Color
0,FCJ5493,NY,PAS,20,BLACK
1,63540MC,NY,COM,46,BRN
2,GCY4187,NY,SRF,21,BLUE
3,95V6675,TX,PAS,21,SILVR
4,FYM5117,NY,SRF,21,WHITE
5,GFM1421,NY,PAS,40,BLK
6,18972BB,NY,999,14,YELLO
7,WNJ4730,VA,PAS,14,BLK
8,68091JZ,NY,COM,46,WH
9,EWV4127,NY,PAS,21,GREY


In [7]:
# Count number of rows in the archive. This will give us an idea for
# how long it takes to scan the archive.

import time

start_scan = time.perf_counter()

rows = db.stream('vehicle_data').count()

end_scan = time.perf_counter()

print('Scan time {:0.4f} sec. for {} rows.'.format((end_scan - start_scan), rows))

Scan time 256.1054 sec. for 8993835 rows.


## Vehicle Color - Using Functional Dependencies to assist with data standardization


We make use of the fact that 'Plate ID' and 'Registration State' uniquely identify a vehicle. For each vehicle we make the assumption that the color doesn't change within the one year of data that we are looking at here. Thus, the functional dependency 'Plate ID', 'Registration State' -> 'Vehicle Color'. should hold. Violations of that dependency point to different representations of the same color value.

In [1]:
# Open instance for existing database object that maintains the
# dataset and all provenance information.

from openclean.engine.base import DB

db = DB(basedir='.openclean', create=False)

In [2]:
# Create data stream for first dataset snapshot that was commited
# to the database.

ds = db.stream(name='vehicle_data', version=0)

In [3]:
# Get summary of conflicting values in the 'Vehicle Color' attribute.
# The Plate ID and Registration State should identify a vehicle uniquely. We use
# this key to find conflicts in the 'Vehicle Color' column.

from openclean.operator.map.violations import fd_violations

df = ds\
    .select(['Plate ID', 'State', 'Vehicle Color'])\
    .to_df()

groups = fd_violations(df, lhs=['Plate ID', 'State'], rhs='Vehicle Color')
conflicts = groups.summarize_conflicts('Vehicle Color')

In [4]:
# Print summary for conflicting values.

conflicts.most_common(50)

[('GY', 170831),
 ('BLACK', 157776),
 ('GREY', 148847),
 ('WHITE', 135127),
 ('BK', 134722),
 ('WH', 122401),
 ('SILVE', 108223),
 ('BLUE', 79173),
 ('BL', 64820),
 ('OTHER', 52313),
 ('RED', 46660),
 ('BLK', 41022),
 ('GRAY', 38376),
 ('RD', 38226),
 ('TAN', 35244),
 ('GREEN', 34815),
 ('GR', 32131),
 ('GOLD', 24429),
 ('TN', 21603),
 ('WHT', 19089),
 ('GRY', 18118),
 ('BROWN', 15000),
 ('YELLO', 11518),
 ('BR', 10562),
 ('YW', 9421),
 ('GL', 8567),
 ('WT', 7496),
 ('MR', 5054),
 ('BLU', 4017),
 ('GRN', 3974),
 ('PURPL', 3932),
 ('SIL', 3596),
 ('SL', 3235),
 ('SILVR', 2822),
 ('ORANG', 2577),
 ('WHI', 2490),
 ('GN', 2441),
 ('LTG', 2344),
 ('SILV', 1972),
 ('BRN', 1958),
 ('LT/', 1834),
 ('DK/', 1393),
 ('OR', 1251),
 ('LTGY', 1124),
 ('BN', 1108),
 ('MAROO', 1100),
 ('DKG', 1097),
 ('BEIGE', 1091),
 ('LAVEN', 1090),
 ('W', 991)]

In [5]:
# Print list of values that occur in a conflict with 'BL'.

conflicts['BL']

ValueConflicts(count=64820, values=Counter({'BLUE': 44044, 'BLACK': 10480, 'BK': 4878, 'GREY': 4673, 'GY': 4020, 'OTHER': 3480, 'BLK': 2788, 'GREEN': 2324, 'BLU': 1813, 'SILVE': 1736, 'GRAY': 1357, 'WHITE': 1330, 'WH': 1296, 'GR': 1244, 'BROWN': 554, 'GRY': 540, 'RD': 495, 'RED': 471, 'BR': 456, 'TAN': 316, 'TN': 281, 'PURPL': 250, 'GRN': 228, 'GOLD': 221, 'BRN': 180, 'WHT': 170, 'GN': 149, 'BN': 139, 'GL': 103, 'BRWN': 93, 'MR': 86, 'BRW': 80, 'DKB': 73, 'BLGY': 65, 'WT': 59, 'ORANG': 54, 'DK/': 53, 'BRO': 50, 'B': 50, 'YELLO': 50, 'SILVR': 49, 'PR': 47, 'BWN': 45, 'SL': 44, 'SIL': 41, 'SILV': 39, 'BW': 39, 'DKBL': 37, 'YW': 37, 'LAVEN': 34, 'LTB': 33, 'G': 31, 'BLBL': 30, 'LT/': 30, 'LTG': 29, 'BROW': 28, 'BLGR': 27, 'OR': 27, 'MULTI': 24, 'BLE': 22, 'BLRD': 18, 'NAVY': 18, 'BLG': 17, 'TEAL': 17, 'BLCK': 15, 'MAROO': 14, 'BURG': 13, 'BL/': 13, 'BLTN': 13, 'DKG': 12, 'BL-GY': 12, 'BM': 12, 'BEIGE': 11, 'BI': 11, 'LTGY': 11, 'BRONW': 10, 'LTBL': 10, 'DKBLU': 10, 'W': 10, 'BLV': 10, 'SL

### First Iteration

In [6]:
# Define initial mapping for color standardization.

mapping = {
    'GRY': 'GRAY',
    'GY': 'GRAY',
    'GY/': 'GRAY',
    'GYGY': 'GRAY',
    'GREY': 'GRAY',
    'SILVE': 'SILVER',
    'SILV': 'SILVER',
    'SILVR': 'SILVER',
    'SLV': 'SILVER',
    'W': 'WHITE',
    'WH': 'WHITE',
    'WH/': 'WHITE',
    'WHT': 'WHITE',
    'WT': 'WHITE',
    'WHI': 'WHITE',
    'BK': 'BLACK',
    'BLK': 'BLACK',
    'RD': 'RED',
    'RD/': 'RED',
    'TN': 'TAN',
    'GL': 'GOLD',
    'GD': 'GOLD',
    'GLD': 'GOLD',
    'GR': 'GREEN',
    'GRN': 'GREEN',
    'GN': 'GREEN',
    'BLU': 'BLUE',
    'YW': 'YELLOW',
    'YL': 'YELLOW',
    'YELLO': 'YELLOW',
    'YELL': 'YELLOW',
    'YEL': 'YELLOW',
    'BN': 'BROWN',
    'BR': 'BROWN',
    'BRN': 'BROWN',
    'BRWN': 'BROWN',
    'BRW': 'BROWN',
    'BRO': 'BROWN',
    'BRON': 'BROWN',
    'BROW': 'BROWN',
    'ORANG': 'ORANGE',
    'OR': 'ORANGE',
    'MAROO': 'MAROON',
    'MR': 'MAROON',
    'MARRO': 'MAROON',
    'MA': 'MAROON',
    'MAR': 'MAROON',
    'MARO': 'MAROON',
    'MARON': 'MAROON',
    'MAIOO': 'MAROON',
    'MARN': 'MAROON',
    'PURPL': 'PURPLE',
    'LAVEN': 'LAVENDER'
}

In [7]:
# Apply update operation on current dataset snapshot in a first
# iteration for standardizing color values.

import time

from openclean.function.value.mapping import Standardize
from openclean.operator.transform.update import Update

start_update = time.perf_counter()

db.apply(name='vehicle_data', operations=Update('Vehicle Color', Standardize(mapping)))

end_update = time.perf_counter()

print('Update time {:0.4f} sec.'.format(end_update - start_update))

Update time 895.8141 sec.


In [8]:
# Print first ten rows of the modified dataset snapshot.

db.stream('vehicle_data').head()

,Plate ID,State,Plate Type,Violation Code,Vehicle Color
0,FCJ5493,NY,PAS,20,BLACK
1,63540MC,NY,COM,46,BROWN
2,GCY4187,NY,SRF,21,BLUE
3,95V6675,TX,PAS,21,SILVER
4,FYM5117,NY,SRF,21,WHITE
5,GFM1421,NY,PAS,40,BLACK
6,18972BB,NY,999,14,YELLOW
7,WNJ4730,VA,PAS,14,BLACK
8,68091JZ,NY,COM,46,WHITE
9,EWV4127,NY,PAS,21,GRAY


In [9]:
# Get count for distinct colors in the dataset.

colors = db.stream('vehicle_data').distinct('Vehicle Color')

In [10]:
len(colors)

2769

In [11]:
colors.most_common(50)

[('WHITE', 2592600),
 ('GRAY', 1718237),
 ('BLACK', 1669913),
 ('RED', 536941),
 ('BL', 441801),
 ('GREEN', 362989),
 ('BROWN', 360793),
 ('SILVER', 320403),
 ('BLUE', 305936),
 ('TAN', 202539),
 ('YELLOW', 115890),
 ('GOLD', 106078),
 ('OTHER', 91292),
 ('MAROON', 33238),
 ('ORANGE', 24231),
 ('LTG', 13094),
 ('PURPLE', 7691),
 ('LT/', 7191),
 ('SL', 6985),
 ('DK/', 6170),
 ('SIL', 6070),
 ('DKG', 5953),
 ('PR', 4769),
 ('DKB', 3251),
 ('LTGY', 2217),
 ('BLG', 1599),
 ('BEIGE', 1585),
 ('LAVENDER', 1548),
 ('LTB', 1501),
 ('DKR', 1391),
 ('NOC', 1250),
 ('BURG', 1128),
 ('DKGY', 919),
 ('WHB', 832),
 ('WHBL', 745),
 ('SLVR', 739),
 ('BL/', 698),
 ('WHG', 691),
 ('R', 595),
 ('BG', 588),
 ('DKBL', 574),
 ('LTT', 540),
 ('BKGY', 539),
 ('G', 507),
 ('BWN', 506),
 ('BKG', 496),
 ('BURGU', 489),
 ('BLW', 444),
 ('WHO', 390),
 ('WHIT', 379)]

### Second Iteration

In [1]:
# Open instance for existing database object that maintains the
# dataset and all provenance information.

from openclean.engine.base import DB

db = DB(basedir='.openclean', create=False)

In [2]:
# Get remaining conflicts.

from openclean.operator.map.violations import fd_violations

df = db.stream('vehicle_data')\
    .select(['Plate ID', 'State', 'Vehicle Color'])\
    .to_df()

groups = fd_violations(df, lhs=['Plate ID', 'State'], rhs='Vehicle Color')

In [3]:
len(groups.keys())

369163

In [4]:
# Show examples for vehicles that still occur in the dataset with
# different colors.

for key in list(groups.keys())[:10]:
    print(groups.get(key))
    print('\n')

        Plate ID State Vehicle Color
3780362  GEL1905    NY          BLUE
3819284  GEL1905    NY             G
4483970  GEL1905    NY          GRAY
4483975  GEL1905    NY          GRAY


        Plate ID State Vehicle Color
5699758  11JT165    AL         WHITE
8394987  11JT165    AL          BLUE


       Plate ID State Vehicle Color
284155  GGF4261    NY           SIL
321441  GGF4261    NY          GRAY
667157  GGF4261    NY          GRAY
704459  GGF4261    NY        SILVER


        Plate ID State Vehicle Color
1941732  FZY8582    NY          BLUE
2712519  FZY8582    NY            BL
3345735  FZY8582    NY            BL
3400502  FZY8582    NY            BL
3449810  FZY8582    NY          BLUE
5553833  FZY8582    NY            BL
7817526  FZY8582    NY            BL


        Plate ID State Vehicle Color
174342   FWF4823    NY            BL
477162   FWF4823    NY            BL
990484   FWF4823    NY            BL
4793485  FWF4823    NY            BL
5256268  FWF4823    NY          BLU

In [5]:
# Resolve conflicts for groups where exactly one value from a set of
# ground truth values (i.e., valid colors) occurs with other values
# that are not in the ground truth set. In these cases we replace all
# color values in the group with the value from the ground truth set.


COLORS = {
    'WHITE',
    'GRAY',
    'BLACK',
    'RED',
    'GREEN',
    'BROWN',
    'SILVER',
    'BLUE',
    'TAN',
    'YELLOW',
    'GOLD',
    'MAROON',
    'ORANGE',
    'PURPLE',
    'BEIGE',
    'LAVENDER'
}


group_mapping = dict()

for key in groups.keys():
    values = groups.values(key=key, columns='Vehicle Color')
    gt_values = [v for v in values if v in COLORS]
    if len(gt_values) == 1:
        group_mapping[key] = gt_values[0]

In [6]:
print('{} groups in mapping.\n'.format(len(group_mapping)))

for key in list(group_mapping.keys())[:10]:
    print('{} = {}'.format(key, group_mapping[key]))
print('...')

95432 groups in mapping.

('FZY8582', 'NY') = BLUE
('FWF4823', 'NY') = BLUE
('DLY8648', 'NY') = BLUE
('FZN7296', 'NY') = BLUE
('FMA4240', 'NY') = GREEN
('FBK8839', 'NY') = BLUE
('FXM3011', 'NY') = BLUE
('BRG4605', 'NY') = GRAY
('FSJ4208', 'NY') = BLUE
('EXR5099', 'NY') = GRAY
...


In [7]:
# Look at one of the groups. These examples show already
# some potential limitations of the approach, though.

groups.get(('FMA4240', 'NY'))

,Plate ID,State,Vehicle Color
141666,FMA4240,NY,BL
5039920,FMA4240,NY,BL
5699406,FMA4240,NY,GREEN


In [8]:
groups.get(('FZY8582', 'NY'))

,Plate ID,State,Vehicle Color
1941732,FZY8582,NY,BLUE
2712519,FZY8582,NY,BL
3345735,FZY8582,NY,BL
3400502,FZY8582,NY,BL
3449810,FZY8582,NY,BLUE
5553833,FZY8582,NY,BL
7817526,FZY8582,NY,BL


In [9]:
groups.get(('EXR5099', 'NY'))

,Plate ID,State,Vehicle Color
654630,EXR5099,NY,BL
7584957,EXR5099,NY,GRAY


In [10]:
# Apply update operation on current dataset snapshot in a first
# iteration for standardizing color values.

import time

from openclean.function.eval.base import Col
from openclean.function.eval.mapping import Lookup
from openclean.operator.transform.update import Update

start_update = time.perf_counter()

func = Lookup(columns=['Plate ID', 'State'], mapping=group_mapping, default=Col('Vehicle Color'))
db.apply(name='vehicle_data', operations=Update('Vehicle Color', func=func))

end_update = time.perf_counter()

print('Update time {:0.4f} sec.'.format(end_update - start_update))

Update time 843.5912 sec.


In [11]:
# Get updated list of distinct colors and their frequency
# in the Vehicle Color column of the latest dataset snapshot.

colors = db.stream('vehicle_data').distinct('Vehicle Color')

In [12]:
len(colors)

2109

In [14]:
colors.most_common(25)

[('WHITE', 2605597),
 ('GRAY', 1748009),
 ('BLACK', 1697283),
 ('RED', 545345),
 ('BLUE', 437059),
 ('GREEN', 371324),
 ('BROWN', 362548),
 ('SILVER', 327988),
 ('BL', 276686),
 ('TAN', 205121),
 ('YELLOW', 116515),
 ('GOLD', 107407),
 ('OTHER', 59468),
 ('MAROON', 33813),
 ('ORANGE', 24557),
 ('LTG', 8903),
 ('PURPLE', 8476),
 ('LT/', 4974),
 ('SL', 4969),
 ('SIL', 3969),
 ('DK/', 3814),
 ('DKG', 3645),
 ('PR', 3360),
 ('DKB', 1800),
 ('BEIGE', 1613)]